In [1]:
## La version de python utilisée est 3.12.7

!pip install -r requirements.txt -q

In [2]:
import pandas as pd
import requests
import lxml as lxml
from bs4 import BeautifulSoup
import io as io
import math
import gzip
import shutil
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import json
from pandasgui import show
import numpy as np
from io import BytesIO
from folium.plugins import HeatMap
import nbconvert
from shapely.geometry import Point
import overpy
import ast
from decimal import Decimal
from pandasgui import show
import zipfile
import shutil

from script import geolocaliser
from script import process_data
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

---
# Partie I : Recenser les communes cotières et leurs attributs
---

In [3]:
# Chargement du shapefile dans un geodataframe et suppression des communes doublons
shapefile_path = "data/communes_cotieres/communes_cotieres.shp"

df_cotieres = gpd.read_file(shapefile_path)

df_cotieres = df_cotieres[['code', 'nom', 'NumDep']].drop_duplicates(subset='nom').sort_values(by='nom').reset_index(drop=True)

1. Création du fichier des communes cotières

In [4]:
liste_cotieres = df_cotieres['nom'].unique().tolist()

In [5]:
df_cotieres = df_cotieres.rename(columns={
    'nom': 'Nom commune',
    'code': 'code_commune',
    'NumDep': 'departement'
})

region = pd.read_csv('data/code_region.csv',encoding="utf8",sep=";")

df_cotieres['departement'] = df_cotieres['departement'].astype(str).astype(int)

df_cotieres = df_cotieres.merge(
    region,  # DataFrame region utilisé pour la fusion
    left_on='departement', 
    right_on='departmentCode',  # Colonne correspondante dans region
    how='left'  # Fusion de type left join
)

In [6]:
url = "https://www.insee.fr/fr/statistiques/fichier/7739582/ensemble.zip"
zip_path = "ensemble.zip"
extracted_folder = "ensemble"

# Télécharger le fichier ZIP
response = requests.get(url)
with open(zip_path, 'wb') as file:
    file.write(response.content)

# Dézipper le fichier ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

# Charger le fichier CSV dans un DataFrame
csv_path = os.path.join(extracted_folder, "donnees_communes.csv")
df_pop = pd.read_csv(csv_path,sep=";",encoding="utf8")

os.remove("ensemble.zip")
shutil.rmtree("ensemble")

df_pop = df_pop[['COM',"PMUN"]].rename(columns={'COM':'code_commune',
                                                'PMUN':'Population'})

# Afficher les premières lignes du DataFrame
print(df_pop.head())

df_cotieres = df_cotieres.merge(df_pop,left_on='code_commune',right_on='code_commune')


  code_commune  Population
0        01001         832
1        01002         267
2        01004       14854
3        01005        1897
4        01006         113


In [7]:
stats_region_filtered = df_cotieres.groupby('regionName').apply(
    lambda x: pd.Series({
        'nombre_communes': x['Nom commune'].nunique(),
        'population_totale': x['Population'].sum()  # Décommente si la colonne 'Population' existe
    })
).reset_index()

# Afficher les résultats
tableau_communes = stats_region_filtered.sort_values(by='nombre_communes', ascending=False).fillna("").style.hide(axis="index")

# Afficher le tableau sans NaN et sans index
tableau_communes

C:\Users\flori\AppData\Local\Temp\ipykernel_12636\3091059007.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stats_region_filtered = df_cotieres.groupby('regionName').apply(


regionName,nombre_communes,population_totale
Bretagne,288,1374425
Normandie,238,1058566
Nouvelle-Aquitaine,198,1277130
Corse,101,282905
Pays de la Loire,79,975375
Occitanie,71,749619
Provence-Alpes-Côte d'Azur,64,1737225
Hauts-de-France,58,419583


In [8]:
# Calcul des statistiques par commune
stats_communes = df_cotieres.groupby('Nom commune').apply(
    lambda x: pd.Series({
        'Population': x['Population'].sum()  # Décommente si la colonne 'Population' existe
    })
).reset_index()

top_20_communes = (
    stats_communes
    .sort_values(by='Population', ascending=False)
    .head(20)
    .style.hide(axis="index")
    .format({'Population': lambda x: f"{x:,.0f}".replace(',', ' ')})  # Remplacement de la virgule par un espace
)


top_20_communes


C:\Users\flori\AppData\Local\Temp\ipykernel_12636\296672310.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stats_communes = df_cotieres.groupby('Nom commune').apply(


Nom commune,Population
Nice,348 085
Nantes,323 204
Bordeaux,261 804
Toulon,180 452
Havre,166 058
Brest,139 619
Perpignan,119 656
Rouen,114 083
Caen,108 200
Dunkerque,86 788


In [10]:
# Charger le fichier GeoJSON des communes de France depuis le lien GitHub
url_commune = 'https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/communes.geojson'
response_commune = requests.get(url_commune)
communes_geojson = response_commune.json()

# Charger le GeoJSON dans un GeoDataFrame
gdf_communes = gpd.GeoDataFrame.from_features(communes_geojson['features'])
gdf_communes_cotieres = gdf_communes[gdf_communes['nom'].isin(liste_cotieres)]
gdf_communes_cotieres = gdf_communes_cotieres.merge(df_cotieres[['Nom commune', 'Population']], 
                                                    left_on='nom', 
                                                    right_on='Nom commune'
                                                    ).drop(columns=['Nom commune'])


In [11]:
# Calculer les centroïdes pour chaque commune
gdf_communes_cotieres['centroid'] = gdf_communes_cotieres.geometry.centroid
gdf_communes_cotieres['latitude_centre'] = gdf_communes_cotieres['centroid'].y
gdf_communes_cotieres['longitude_centre'] = gdf_communes_cotieres['centroid'].x
gdf_communes_cotieres = gdf_communes_cotieres.drop(columns=['centroid'])

liste_commune = gdf_communes_cotieres['nom'].to_list()

On utilise l'API Overpass pour successivement géolocaliser les mairies, les stations de transport et les plages de la commune.

In [11]:
# Appliquer la fonction sur chaque commune
api = overpy.Overpass()

gdf_communes_cotieres[['latitude_mairie', 'longitude_mairie']] = gdf_communes_cotieres.apply(
    lambda row: pd.Series(geolocaliser.get_townhall_coordinates(row['nom'], api)),
    axis=1
)

Erreur lors de la requête Overpass : 'Way' object has no attribute 'center'
Erreur lors de la requête Overpass : 'Way' object has no attribute 'center'
Erreur lors de la requête Overpass : 'Way' object has no attribute 'center'
Erreur lors de la requête Overpass : 'Way' object has no attribute 'center'
Erreur lors de la requête Overpass : 'Way' object has no attribute 'center'
Erreur lors de la requête Overpass : 'Way' object has no attribute 'center'
Erreur lors de la requête Overpass : 'Way' object has no attribute 'center'
Erreur lors de la requête Overpass : 'Way' object has no attribute 'center'
Erreur lors de la requête Overpass : 'Way' object has no attribute 'center'
Erreur lors de la requête Overpass : 'Way' object has no attribute 'center'
Erreur lors de la requête Overpass : 'Way' object has no attribute 'center'
Erreur lors de la requête Overpass : 'Way' object has no attribute 'center'
Erreur lors de la requête Overpass : 'Way' object has no attribute 'center'
Erreur lors 

Dans le cas où Overpass ne permet pas de géolocaliser la mairie, on fait une recherche par mot-clé à partir de l'API Adresse de l'Institut National Géographique (ING)

In [12]:
import importlib
importlib.reload(geolocaliser)

# Liste des mots-clés pour localiser les mairies
mots_cles = [
    "Mairie+de", 
    "Hotel+de+ville+de", 
    "Bureau+du+Maire+de", 
    "Salle+des+Fêtes+de", 
    "Maison+Communale+de", 
    "Centre+Administratif+de", 
    "Hôtel+de+Ville", 
    "Bâtiment+Municipal+de", 
    "Mairie+municipale+de", 
    "Maison+des+Services+de"
]

# Compléter les coordonnées manquantes
gdf_communes_cotieres = geolocaliser.geolocaliser_mot_cle(
    gdf_communes_cotieres, 
    colonne_commune='nom', 
    colonne_geometry='geometry', 
    mots_cles=mots_cles, 
    colonne_latitude='latitude_mairie', 
    colonne_longitude='longitude_mairie'
)

In [12]:
import importlib
# Recharge le module
importlib.reload(geolocaliser)

# Liste des mots-clés pour localiser les plages
mots_cles_plages = [ 
    "Plage+de",
    "Plage",
    "Beach"
]

# Compléter les coordonnées manquantes pour les plages
gdf_communes_cotieres = geolocaliser.geolocaliser_mot_cle(
    gdf_communes_cotieres, 
    colonne_commune='nom', 
    colonne_geometry='geometry', 
    mots_cles=mots_cles_plages, 
    colonne_latitude='latitude_plage', 
    colonne_longitude='longitude_plage',
    mode="all"
)

In [14]:
show(gdf_communes_cotieres)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
C:\Users\flori\AppData\Local\Temp\ipykernel_12636\717718677.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  show(gdf_communes_cotieres)
C:\Users\flori\AppData\Local\Temp\ipykernel_12636\717718677.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  show(gdf_communes_cotieres)
C:\Users\flori\AppData\Local\Temp\ipykernel_12636\717718677.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[

In [13]:
# Pour les plages
api = overpy.Overpass()

gdf_communes_cotieres['beach_coordinates'] = gdf_communes_cotieres.apply(
    lambda row: geolocaliser.get_beach_coordinates(row['nom'], api),
    axis=1
)

In [91]:
# Pour les stations de transport
api = overpy.Overpass()

gdf_communes_cotieres['station'] = gdf_communes_cotieres.apply(
    lambda row: geolocaliser.get_station_coordinates(row['nom'], api),
    axis=1
)

# Liste des mots-clés pour localiser les gares
mots_cles_gares = [ 
    "Gare+de"
]

# Compléter les coordonnées manquantes pour les pgares
gdf_communes_cotieres = geolocaliser.geolocaliser_mot_cle(
    gdf_communes_cotieres, 
    colonne_commune='nom', 
    colonne_geometry='geometry', 
    mots_cles=mots_cles_gares, 
    colonne_latitude='latitude_gare', 
    colonne_longitude='longitude_gare'
)

In [92]:
# Utilisation de la fonction avec un GeoDataFrame
api = overpy.Overpass()

gdf_communes_cotieres[['latitude_port', 'longitude_port']] = gdf_communes_cotieres.apply(
    lambda row: pd.Series(geolocaliser.get_ports(row['nom'], api)),
    axis=1
)

# Liste des mots-clés pour localiser les plages
mots_cles_ports = [ 
    "Port+de",
]

# Compléter les coordonnées manquantes pour les ports
gdf_communes_cotieres = geolocaliser.geolocaliser_mot_cle(
    gdf_communes_cotieres, 
    colonne_commune='nom', 
    colonne_geometry='geometry', 
    mots_cles=mots_cles_ports, 
    colonne_latitude='latitude_port', 
    colonne_longitude='longitude_port'
)

Erreur de requête pour la commune : Port-Jérôme-sur-Seine, statut : 504


In [93]:
# Application
gdf_communes_cotieres['station'] = gdf_communes_cotieres['station'].apply(process_data.fix_coordinates_format)
gdf_communes_cotieres['beach_coordinates'] = gdf_communes_cotieres['beach_coordinates'].apply(process_data.fix_coordinates_format)

In [94]:
gdf_communes_cotieres["beach_coordinates"] = gdf_communes_cotieres.apply(
    lambda row: row["beach_coordinates"] + [(row["latitude_plage"], row["longitude_plage"])] if row["latitude_plage"] is not None and row["longitude_plage"] is not None else row["beach_coordinates"],
    axis=1
)

In [95]:
# Calcul de la part de chaque variable renseignée
summary = {
    "Variable": gdf_communes_cotieres.columns,
    "Part (%)": [
        gdf_communes_cotieres[col].notnull().mean() * 100 
        for col in gdf_communes_cotieres.columns
    ]
}

# Création d'un DataFrame pour présentation
summary_df = pd.DataFrame(summary)

# Filtrer uniquement les colonnes désirées
columns_to_display = [
    "geometry", "Population", "latitude_mairie", 
    "beach_coordinates", "station", "latitude_port"
]
filtered_summary_df = summary_df[summary_df["Variable"].isin(columns_to_display)]

# Style pour affichage dans le notebook
styled_table = (
    filtered_summary_df.style
    .hide(axis="index")  # Cache l'index
    .set_caption("Part de variables renseignées")
    .format({"Part (%)": "{:.1f}%"})  # Formate les pourcentages
)

# Affichage dans le notebook
styled_table

Variable,Part (%)
geometry,100.0%
Population,100.0%
latitude_mairie,90.6%
beach_coordinates,100.0%
station,100.0%
latitude_port,58.8%


In [96]:
gdf_communes_cotieres = gdf_communes_cotieres.drop(columns=['latitude_plage','longitude_plage','latitude_gare', 'longitude_gare','code'])

In [97]:
gdf_communes_cotieres.to_csv('data/communes_cotieres.csv',encoding="utf8",sep=";",index=False)